In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [3]:
labels = df[1].values

In [4]:
type(labels)

numpy.ndarray

In [5]:
texts = df[0].values.tolist()

In [6]:
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification

In [7]:
tokenizer = RobertaTokenizer.from_pretrained("siebert/sentiment-roberta-large-english")

C:\Users\yong\anaconda3\envs\textmining_cpu\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yong\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [8]:
from tensorflow.keras.utils import to_categorical
y_one_hot = to_categorical(labels)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(texts, y_one_hot, test_size=0.2, random_state=0)

In [10]:
X_train_tokenized = tokenizer(X_train, return_tensors="np", max_length=30, padding='max_length', truncation=True)
X_test_tokenized = tokenizer(X_test, return_tensors="np", max_length=30, padding='max_length', truncation=True)

In [11]:
model = TFRobertaForSequenceClassification.from_pretrained("siebert/sentiment-roberta-large-english", num_labels=2)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at siebert/sentiment-roberta-large-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [12]:
optimizer = tf.keras.optimizers.Adam(2e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [13]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
checkpoint_filepath = "./checkpoints/checkpoint_roberta_en_v2"
mc = ModelCheckpoint(checkpoint_filepath, monitor='val_loss', mode='min', 
                     save_best_only=True, save_weights_only=True)

In [14]:
model.fit(dict(X_train_tokenized), y_train, epochs=10, batch_size=128, 
          validation_split=0.1, callbacks=[es, mc])

Epoch 1/10
39/39 [==============================] - 749s 18s/step - loss: 0.2354 - accuracy: 0.9171 - val_loss: 0.2123 - val_accuracy: 0.9314
Epoch 2/10
39/39 [==============================] - 666s 17s/step - loss: 0.1675 - accuracy: 0.9414 - val_loss: 0.2221 - val_accuracy: 0.9332
Epoch 3/10
39/39 [==============================] - 708s 18s/step - loss: 0.1295 - accuracy: 0.9552 - val_loss: 0.2121 - val_accuracy: 0.9404
Epoch 4/10
39/39 [==============================] - 666s 17s/step - loss: 0.0889 - accuracy: 0.9701 - val_loss: 0.2212 - val_accuracy: 0.9404
Epoch 5/10
39/39 [==============================] - 666s 17s/step - loss: 0.0578 - accuracy: 0.9797 - val_loss: 0.2529 - val_accuracy: 0.9368
Epoch 6/10
39/39 [==============================] - 668s 17s/step - loss: 0.0500 - accuracy: 0.9864 - val_loss: 0.3065 - val_accuracy: 0.9314
Epoch 6: early stopping


In [15]:
model.load_weights(checkpoint_filepath)

In [16]:
model.evaluate(dict(X_test_tokenized), np.array(y_test))

44/44 [==============================] - 70s 2s/step - loss: 0.2194 - accuracy: 0.9306


[0.2193983942270279, 0.9306358098983765]

In [17]:
y_preds = model.predict(dict(X_test_tokenized))
prediction_probs = tf.nn.softmax(y_preds.logits,axis=1).numpy()
y_predictions = np.argmax(prediction_probs, axis=1)
y_test = np.argmax(y_test, axis=1)
from sklearn.metrics import classification_report
print(classification_report(y_predictions, y_test))

44/44 [==============================] - 70s 2s/step
              precision    recall  f1-score   support

           0       0.91      0.94      0.92       622
           1       0.95      0.92      0.94       762

    accuracy                           0.93      1384
   macro avg       0.93      0.93      0.93      1384
weighted avg       0.93      0.93      0.93      1384

